In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import pandas as pd

In [4]:
import tqdm as notebook_tqdm
import matplotlib.pyplot as plt

In [5]:
sys.path.append("CheXbert_task2/src")

In [ ]:
import CheXbert_task2
import CheXbert_task2.src
import CheXbert_task2.src.models.bert_labeler
import CheXbert_task2.src.models as models
from CheXbert_task2.src import *
from CheXbert_task2.src.utils import test

In [7]:
from CheXbert_task2.src.run_bert import *
from CheXbert_task2.src.run_bert import model_from_ckpt

### Variables

In [9]:
train_csv_path = #path to train set
dev_csv_path = #path to dev set
train_imp_path = #path to train impressions
dev_imp_path = #path to dev impressions
out_path = #path to where you want the model to be saved
checkpoint_path = #specify checkpoint path if desired

In [107]:
#run if you don't want pre trained weights
checkpoint_path = None

In [ ]:
model, optimizer, device = model_from_ckpt(bert_labeler(p=0), checkpoint_path)
optimizer = None

In [11]:
batch_size = 10

In [85]:
#run to perform reweighting
train_set = pd.read_csv(train_csv_path)
y_train = np.zeros(train_set.shape[0])
for i in range(train_set.shape[0]):
    row = train_set.iloc[i]
    for idx,cond in enumerate(CONDITIONS):
        if row[cond]==1:
            y_train[i]=int(idx)
y_train= y_train.astype(int)
weight_train = []
for cond in CONDITIONS:
    mean = train_set[cond].mean()
    weight_train.append(1/mean)
samples_weight = np.array([weight_train[t] for t in y_train])
samples_weight = torch.from_numpy(samples_weight)

In [12]:
#dataloaders
dataloaders = load_data(train_csv_path, train_imp_path,
                        dev_csv_path, dev_imp_path, batch_size = batch_size)

In [88]:
#dataloaders with resampling (run to perform reweighting)
dataloaders = load_data(train_csv_path, train_imp_path, dev_csv_path, dev_imp_path,
                        batch_size = batch_size, train_weights=samples_weight)

In [13]:
## model parameters
save_path = out_path
dataloaders=dataloaders
model = model
optimizer = optimizer
device = device
log_every = 1
valid_niter = 20
lr = 0.00002
log_every = 5
num_epochs = 20

### Training

In [ ]:
res= train(save_path, dataloaders, weights = np.ones(len(CONDITIONS)), f1_weights= None, model=model, device=device,
          optimizer=optimizer, lr=lr, log_every=log_every,
          valid_niter=valid_niter, best_metric=0.5, num_epochs = num_epochs)

### Plotting loss and metrics

In [15]:
loss_hist = res[0]
loss_hist_train = res[7]
acc_values = res[2]
F1_values = res[4]
acc_values_train = res[3]
F1_values_train = res[5]
loss_hist_batch_level = res[6]

In [17]:
def plot_figures(F1_values,acc_values,loss_hist, title):
    plt.plot([i for i in range(len(loss_hist))], loss_hist)
    plt.xlabel("iterations")
    plt.ylabel("loss")
    plt.title(title)
    plt.show()
    for key, data_list in acc_values.items():
        plt.plot([i for i in range(len(data_list))], data_list, label=key)
    plt.legend()
    plt.xlabel("iteration")
    plt.ylabel("Accuracy")
    plt.title(title)
    plt.show()
    for key, data_list in F1_values.items():
        plt.plot([i for i in range(len(data_list))], data_list, label=key)
    plt.legend()
    plt.xlabel("iteration")
    plt.ylabel("F1")
    plt.title(title)
    plt.show()

In [ ]:
plot_figures(F1_values,acc_values,loss_hist, "Dev")

In [ ]:
plot_figures(F1_values_train,acc_values_train,loss_hist_train, "Train")

In [ ]:
plt.plot([i for i in range(len(loss_hist_batch_level))], loss_hist_batch_level)
plt.xlabel("iterations")
plt.ylabel("loss")
plt.title("Loss values at batch level")
plt.show()

## Evaluation on test set

In [12]:
test_csv_path = #path to test set
test_list_path = #path to test set imps
batch_size = 10

In [ ]:
model, optimizer, device = model_from_ckpt(bert_labeler(p=0), None)
checkpoint_path= #path to checkpoint of the model you wish to evaluate
test_ld=load_test_data(test_csv_path, test_list_path, batch_size=batch_size, 
                   num_workers=NUM_WORKERS, shuffle=False)
test(model, checkpoint_path, test_ld, None)